These are our imports

In [1]:
#imports
import numpy as np
print("numpy imported")
import json
print("json imported")
import time
print("time imported")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
print("tensorflow imported") 
from keras import Sequential
print("keras imported")
import config
channelID,serverID,targetID = config.getScope()
print("config imported")
import pickle
print("pickle imported")
import random
print("random imported")
import string
print("string imported")
from discAutoMsg import discord_automessage
print("Discord Auto Message library imported")


numpy imported
json imported
time imported
tensorflow imported
keras imported
config imported
pickle imported
random imported
string imported
{"token": "MzA4MzAwOTIzMzcxOTEzMjE4.YNP-0Q.H7_KbCQ6eWdNwih8xbPi1ioCuzU", "user_settings": {"locale": "en-US", "theme": "dark"}}
Discord Auto Message library imported


Run the section below to process your data

In [2]:
channelID,serverID,targetID = config.getScope()
daTestDoc = open("TestDoc","r",encoding="utf-8")
daMessageList = eval(daTestDoc.read())
with open('message.list','wb') as messageList_file:
    pickle.dump(daMessageList,messageList_file)
    print(":flushed:")

:flushed:


In [3]:
inputList,outputList = [],[]
with open("message.list",'rb') as daMessageList:
    channelID,serverID,targetID = config.getScope()
    daMessageList = pickle.load(daMessageList)
    for i in range(len(daMessageList)):
        if i != len(daMessageList)+1:
            message = daMessageList[i]
            if message["author"]["id"] == str(targetID):
                #if i != 0:
                if True:
                    previousMessage = daMessageList[i-1]
                    if previousMessage["author"]["id"] == str(targetID):
                        if len(outputList) != 0:    
                            outputList[len(outputList)-1] += "   " + message["content"]
                    else:
                        if type(message.get("referenced_message"))!=dict:
                            outputList.append(message["content"])
                            message = daMessageList[i+1]
                            inputList.append(message["content"])
                        else:
                            inputList.append(message["referenced_message"]["content"])
                            outputList.append(message["content"])

        myIndice = random.randint(0,len(inputList))

    # for message in range(len(inputList)):
    # 	inputList[message] = removePunctuation(removeCaps(inputList[message]))
    with open('input.list','wb') as inputList_file:
        pickle.dump(inputList,inputList_file)
    with open('output.list','wb') as outputList_file:
        pickle.dump(outputList,outputList_file)
    print("Complete! (1/2)")


with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)
        
        removePunctuation = lambda x: ''.join(char for char in x if not char in string.punctuation)
        removeCaps = lambda x: x.lower()

        for message in range(len(inputList)):
            inputList[message] = removePunctuation(removeCaps(inputList[message]))
            outputList[message] = removePunctuation(removeCaps(outputList[message]))
        #inputList = np.array(inputList)
        #outputList = np.array(outputList)
        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)

        print("Complete! (2/2)")

Complete! (1/2)
Complete! (2/2)


In [130]:
with open('input.list','rb') as inputList_file:
    inputList = pickle.load(inputList_file)
    daRandInt = random.randint(0,len(inputList)-1)
    print(inputList[daRandInt])

[[   0    0    0 ...    8   39  287]
 [   0    0    0 ...    7 1100  287]
 [   0    0    0 ...   13 1101   47]
 ...
 [   0    0    0 ...    8  393  270]
 [   0    0    0 ...   24 2788  126]
 [   0    0    0 ...    0   84   18]]


In [2]:
daShape = None
with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)

        daMaxLen = 0

        tokenizer = Tokenizer()
        #print(type(inputList+outputList))
        tokenizer.fit_on_texts(inputList)
        #print(inputList[0:10])
        inputList = tokenizer.texts_to_sequences(inputList)
        outputList = tokenizer.texts_to_sequences(outputList)

        #print(inputList)

        inputList = pad_sequences(inputList)
        outputList = pad_sequences(outputList)

        if inputList.shape[1] < outputList.shape[1]:
            daMaxLen = inputList.shape[1]
        else:
            daMaxLen = outputList.shape[1]

        inputList = pad_sequences(inputList, maxlen = daMaxLen)
        outputList = pad_sequences(outputList, maxlen = daMaxLen)

        daShape = outputList.shape

        inputList = np.expand_dims(inputList,axis=0)
        outputList = np.expand_dims(outputList,axis=0)

        inputList = to_categorical(inputList)
        outputList = to_categorical(outputList)

        inputList = inputList.astype(np.uint8)
        outputList = outputList.astype(np.uint8)

        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)
        print("Complete!")

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

** This is a test to make sure the filters run correctly. **

In [59]:
with open('input.list','rb') as inputList:
    with open('output.list','rb') as outputList:
        inputList = pickle.load(inputList)
        outputList = pickle.load(outputList)
        for message in range(len(inputList)):
            for letter in ("abcdefghijklmnopqrstuvwxyz".upper()+string.punctuation):
                if letter in inputList[message]:
                    print(inputList[message])
                if letter in outputList[message]:
                    print(outputList[message])
print("!")

!
<ipython-input-59-1b838eb49a3f>:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if letter in inputList[message]:
<ipython-input-59-1b838eb49a3f>:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if letter in outputList[message]:


In [6]:
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop

In [7]:
#Create model
model = tf.keras.models.Sequential()

tf.keras.layers.Input(shape=(daShape))
model.add(layers.LSTM(2, activation="tanh", name="layer1"))
model.add(layers.Dense(3, activation="softmax", name="layer2"))
model.add(layers.Dense(4, name="layer3"))
print(":pray:")

:pray:


In [8]:
rmsprop = RMSprop(learning_rate=0.0001)
with open('input.list','rb') as inputList:
    with open('output.list','rb') as outputList:
        inputList = pickle.load(inputList)
        outputList = pickle.load(outputList)
        print(type(inputList),type(outputList))
        model.compile(optimizer = rmsprop, loss = "categorical_crossentropy", metrics = ['accuracy'])
        #View model info


<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [9]:

model.fit(inputList,outputList, epochs = 1000, batch_size = 40)

Stats

In [149]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (LSTM)                multiple                  664       
_________________________________________________________________
layer2 (Dense)               multiple                  0 (unused)
_________________________________________________________________
layer3 (Dense)               multiple                  0 (unused)
Total params: 664
Trainable params: 664
Non-trainable params: 0
_________________________________________________________________


In [26]:
with open('input.list','rb') as inputList:
    with open('output.list','rb') as outputList:
        inputList = pickle.load(inputList)
        outputList = pickle.load(outputList)
        inputWordCounter = 0
        inputCharCounter = 0
        outputWordCounter = 0
        outputCharCounter = 0
        for message in range(len(inputList)):
            for char in inputList[message]:
                if char == " ":
                    inputWordCounter += 1
                inputCharCounter += 1
        for message in range(len(inputList)):
            for char in outputList[message]:
                if char == " ":
                    outputWordCounter += 1
                outputCharCounter += 1
        print("Count of Input Words: " + str(inputWordCounter))
        print("Count of Input Characters: " + str(inputCharCounter))
        print("Count of Output Words: " + str(outputWordCounter))
        print("Count of Output Characters: " + str(outputCharCounter))


Count of Input Words: 15369
Count of Input Characters: 104750
Count of Output Words: 11887
Count of Output Characters: 83382
